[![Open In Colab](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/badge/open-in-colab.svg)](https://colab.research.google.com/github/crunchdao/quickstarters/blob/master/competitions/structural-break/quickstarters/baseline/baseline.ipynb)
[![Open In Kaggle](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/badge/open-in-kaggle.svg)](https://www.kaggle.com/code/crunchdao/structural-break-baseline)

![Banner](https://raw.githubusercontent.com/crunchdao/quickstarters/refs/heads/master/competitions/structural-break/assets/banner.webp)

# ADIA Lab Structural Break Challenge

## Challenge Overview

Welcome to the ADIA Lab Structural Break Challenge! In this challenge, you will analyze univariate time series data to determine whether a structural break has occurred at a specified boundary point.

### What is a Structural Break?

A structural break occurs when the process governing the data generation changes at a certain point in time. These changes can be subtle or dramatic, and detecting them accurately is crucial across various domains such as climatology, industrial monitoring, finance, and healthcare.

![Structural Break Example](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/competitions/structural-break/quickstarters/baseline/images/example.png)

### Your Task

For each time series in the test set, you need to predict a score between `0` and `1`:
- Values closer to `0` indicate no structural break at the specified boundary point;
- Values closer to `1` indicate a structural break did occur.

### Evaluation Metric

The evaluation metric is [ROC AUC (Area Under the Receiver Operating Characteristic Curve)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html), which measures the performance of detection algorithms regardless of their specific calibration.

- ROC AUC around `0.5`: No better than random chance;
- ROC AUC approaching `1.0`: Perfect detection.

# Setup

The first steps to get started are:
1. Get the setup command
2. Execute it in the cell below

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Reveal token](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/reveal-token.gif)

In [ ]:
# Install the Crunch CLI
%pip install --upgrade crunch-cli

# Setup your local environment
!crunch setup --notebook structural-break hello --token aaaabbbbccccddddeeeeffff

# Your model

## Setup

In [ ]:
import os
import typing

# Import your dependencies
import joblib
import pandas as pd
import scipy
import sklearn.metrics

In [ ]:
import crunch

# Load the Crunch Toolings
crunch = crunch.load_notebook()

## Understanding the Data

The dataset consists of univariate time series, each containing ~2,000-5,000 values with a designated boundary point. For each time series, you need to determine whether a structural break occurred at this boundary point.

The data was downloaded when you setup your local environment and is now available in the `data/` directory.

In [ ]:
# Load the data simply
X_train, y_train, X_test = crunch.load_data()

### Understanding `X_train`

The training data is structured as a pandas DataFrame with a MultiIndex:

**Index Levels:**
- `id`: Identifies the unique time series
- `time`: (arbitrary) The time step within each time series, which is regularly sampled

**Columns:**
- `value`: The values of the time series at each given time step
- `period`: whether you are in the first part of the time series (`0`), before the presumed break point, or in the second part (`1`), after the break point

In [ ]:
X_train

value  period
id    time                  
0     0     0.001858       0
      1    -0.001664       0
      2    -0.004386       0
      3     0.000699       0
      4    -0.002433       0
...              ...     ...
10000 1890 -0.005903       1
      1891  0.007295       1
      1892  0.003527       1
      1893  0.007218       1
      1894  0.000034       1

[23802099 rows x 2 columns]

### Understanding `y_train`

This is a simple `pandas.Series` that tells if a time series id has a structural break, or not, from the presumed break point on.

**Index:**
- `id`: the ID of the time series

**Value:**
- `structural_breakpoint`: Boolean indicating whether a structural break occurred (`True`) or not (`False`)

In [ ]:
y_train

id
0         True
1         True
2        False
3         True
4        False
         ...  
9996     False
9997      True
9998     False
9999     False
10000     True
Name: structural_breakpoint, Length: 10001, dtype: bool

### Understanding `X_test`

The test data is provided as a **`list` of `pandas.DataFrame`s** with the same format as [`X_train`](#understanding-X_test).

It is structured as a list to encourage processing records one by one, which will be mandatory in the `infer()` function.

In [ ]:
print("Number of datasets:", len(X_test))

Number of datasets: 101


In [ ]:
X_test[0]

value  period
id    time                  
10001 0    -0.020657       0
      1    -0.005894       0
      2    -0.003052       0
      3    -0.000590       0
      4     0.009887       0
...              ...     ...
      2517  0.005084       1
      2518 -0.024414       1
      2519 -0.014986       1
      2520  0.012999       1
      2521 -0.022138       1

[2522 rows x 2 columns]

## Strategy Implementation

There are multiple approaches you can take to detect structural breaks:

1. **Statistical Tests**: Compare distributions before and after the boundary point;
2. **Feature Engineering**: Extract features from both segments for comparison;
3. **Time Series Modeling**: Detect deviations from expected patterns;
4. **Machine Learning**: Train models to recognize break patterns from labeled examples.

The baseline implementation below uses a simple statistical approach: a t-test to compare the distributions before and after the boundary point.

#MY SOLUTION

# Structural Break Detection Pipeline for ADIA Lab Challenge
This notebook implements a complete, deterministic pipeline for structural break detection, following the provided technical plan. It covers feature engineering, tabular and deep learning models, ensembling, and the required train/infer functions for competition submission.

## 1. Environment Setup and Imports
Import all required libraries and set global random seeds for reproducibility.

In [ ]:
# Environment Setup and Imports
SEED = 42
import os, random, numpy as np, pandas as pd
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.tsa.stattools as tsa
import ruptures as rpt
import pycatch22
import pywt
import joblib
from tqdm import tqdm
import lightgbm as lgb
import torch
import torch.nn as nn
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score, log_loss, average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

## 2. Dataset Loading and Schema Validation
Load the training parquet files and validate the MultiIndex structure. Convert to per-series dictionaries for further processing.

In [ ]:
# Dataset Loading and Schema Validation
def read_dataset(X_path, y_path=None):
    X = pd.read_parquet(X_path)
    assert isinstance(X.index, pd.MultiIndex)
    assert set(X.columns) >= {'value', 'period'}
    series_dict = {}
    for id_, group in X.groupby(level='id'):
        time = group.index.get_level_values('time').to_numpy()
        value = group['value'].to_numpy()
        period = group['period'].to_numpy()
        series_dict[id_] = {'time': time, 'value': value, 'period': period}
    y_series = None
    if y_path:
        y_series = pd.read_parquet(y_path)
        if isinstance(y_series, pd.DataFrame):
            y_series = y_series.iloc[:,0]
        y_series = y_series.astype(bool)
    return series_dict, y_series

# Example usage:
# series_dict, y_series = read_dataset('X_train.parquet', 'y_train.parquet')

## 3. Data Preprocessing Pipeline
Implement preprocessing steps: splitting by period, NaN handling, outlier winsorization, detrending, and length normalization for CNN input.

In [ ]:
# Data Preprocessing Pipeline
def preprocess_series(series, L_cnn=512):
    time, value, period = series['time'], series['value'], series['period']
    boundary = np.argmax(period == 1)
    x_pre = value[period == 0]
    x_post = value[period == 1]
    # NaN handling
    x_pre = pd.Series(x_pre).interpolate().fillna(method='bfill').fillna(method='ffill').to_numpy()
    x_post = pd.Series(x_post).interpolate().fillna(method='bfill').fillna(method='ffill').to_numpy()
    n_missing_pre = np.isnan(x_pre).sum()
    n_missing_post = np.isnan(x_post).sum()
    # Winsorization
    def winsorize(x):
        q_low, q_high = np.quantile(x, [0.001, 0.999])
        return np.clip(x, q_low, q_high)
    x_pre_w = winsorize(x_pre)
    x_post_w = winsorize(x_post)
    n_winsorized_pre = (x_pre != x_pre_w).sum()
    n_winsorized_post = (x_post != x_post_w).sum()
    # Detrending
    def detrend(x, t):
        p = np.polyfit(t, x, 1)
        return x - np.polyval(p, t)
    x_pre_dt = detrend(x_pre_w, np.arange(len(x_pre_w)))
    x_post_dt = detrend(x_post_w, np.arange(len(x_post_w)))
    # Length normalization for CNN
    def resample(x, L):
        idx = np.linspace(0, len(x)-1, L)
        return np.interp(idx, np.arange(len(x)), x)
    x_pre_cnn = (resample(x_pre_w, L_cnn) - np.mean(x_pre_w)) / (np.std(x_pre_w) + 1e-8)
    x_post_cnn = (resample(x_post_w, L_cnn) - np.mean(x_post_w)) / (np.std(x_post_w) + 1e-8)
    return {
        'x_pre': x_pre_w, 'x_post': x_post_w,
        'x_pre_dt': x_pre_dt, 'x_post_dt': x_post_dt,
        'x_pre_cnn': x_pre_cnn, 'x_post_cnn': x_post_cnn,
        'n_missing_pre': n_missing_pre, 'n_missing_post': n_missing_post,
        'n_winsorized_pre': n_winsorized_pre, 'n_winsorized_post': n_winsorized_post,
        'len_pre': len(x_pre), 'len_post': len(x_post)
    }

## 4. Feature Engineering - Statistical Features
Extract comprehensive statistical features including descriptive stats, distribution tests, autocorrelation, and frequency domain features.

In [ ]:
# Feature Engineering - Statistical Features
def extract_stat_features(pre, post, pre_dt, post_dt):
    eps = 1e-8
    feats = {}
    # Basic stats
    for name, x, y in [('raw', pre, post), ('detr', pre_dt, post_dt)]:
        for seg, arr in [('pre', x), ('post', y)]:
            feats[f'{name}_mean_{seg}'] = np.mean(arr)
            feats[f'{name}_median_{seg}'] = np.median(arr)
            feats[f'{name}_var_{seg}'] = np.var(arr, ddof=1)
            feats[f'{name}_std_{seg}'] = np.std(arr, ddof=1)
            feats[f'{name}_min_{seg}'] = np.min(arr)
            feats[f'{name}_max_{seg}'] = np.max(arr)
            feats[f'{name}_range_{seg}'] = np.max(arr) - np.min(arr)
            feats[f'{name}_mad_{seg}'] = np.median(np.abs(arr - np.median(arr)))
            feats[f'{name}_skew_{seg}'] = stats.skew(arr)
            feats[f'{name}_kurt_{seg}'] = stats.kurtosis(arr)
            feats[f'{name}_zero_ct_{seg}'] = (arr == 0).sum()
            feats[f'{name}_unique_ct_{seg}'] = np.unique(arr).size
            feats[f'{name}_energy_{seg}'] = np.sum(arr**2)
            # Quantiles
            for q in [0.01,0.05,0.10,0.25,0.5,0.75,0.90,0.95,0.99]:
                feats[f'{name}_q{int(q*100)}_{seg}'] = np.quantile(arr, q)
            feats[f'{name}_iqr_{seg}'] = np.quantile(arr,0.75)-np.quantile(arr,0.25)
        # Comparative features
        for f in ['mean','median','var','std','min','max','range','mad','skew','kurt','energy']:
            feats[f'{name}_{f}_diff'] = feats[f'{name}_{f}_post'] - feats[f'{name}_{f}_pre']
            feats[f'{name}_{f}_ratio'] = feats[f'{name}_{f}_post'] / (feats[f'{name}_{f}_pre'] + eps)
            feats[f'{name}_{f}_absdiff'] = np.abs(feats[f'{name}_{f}_post'] - feats[f'{name}_{f}_pre'])
    # Two-sample tests
    t_stat, p_t = stats.ttest_ind(pre, post, equal_var=False)
    ks_stat, p_ks = stats.ks_2samp(pre, post)
    u_stat, p_u = stats.mannwhitneyu(pre, post, alternative='two-sided')
    levene_stat, p_levene = stats.levene(pre, post, center='median')
    chi2_stat, p_chi2 = stats.chisquare(np.histogram(pre, bins=20)[0], np.histogram(post, bins=20)[0])
    feats.update({
        't_stat': t_stat, 't_logp': -np.log10(p_t+1e-300),
        'ks_stat': ks_stat, 'ks_logp': -np.log10(p_ks+1e-300),
        'u_stat': u_stat, 'u_logp': -np.log10(p_u+1e-300),
        'levene_stat': levene_stat, 'levene_logp': -np.log10(p_levene+1e-300),
        'chi2_stat': chi2_stat, 'chi2_logp': -np.log10(p_chi2+1e-300),
        'wasserstein': stats.wasserstein_distance(pre, post),
        'energy': stats.energy_distance(pre, post)
    })
    # Temporal dependence
    acf_pre = tsa.acf(pre, nlags=10, fft=True)
    acf_post = tsa.acf(post, nlags=10, fft=True)
    feats['acf_lag1_diff'] = acf_post[1] - acf_pre[1]
    feats['acf_lag1_ratio'] = acf_post[1] / (acf_pre[1] + eps)
    # Frequency features
    fft_pre = np.fft.rfft(pre)
    fft_post = np.fft.rfft(post)
    power_pre = np.abs(fft_pre)**2
    power_post = np.abs(fft_post)**2
    feats['fft_dom_freq_pre'] = np.argmax(power_pre)
    feats['fft_dom_freq_post'] = np.argmax(power_post)
    feats['fft_dom_power_diff'] = np.max(power_post) - np.max(power_pre)
    # Spectral entropy
    def spectral_entropy(p):
        p = p / (np.sum(p) + eps)
        return -np.sum(p * np.log(p + eps))
    feats['spectral_entropy_pre'] = spectral_entropy(power_pre)
    feats['spectral_entropy_post'] = spectral_entropy(power_post)
    # Wavelet energy
    coeffs_pre = pywt.wavedec(pre, 'db4', level=3)
    coeffs_post = pywt.wavedec(post, 'db4', level=3)
    for i, (c_pre, c_post) in enumerate(zip(coeffs_pre, coeffs_post)):
        feats[f'wavelet_energy_pre_l{i}'] = np.sum(c_pre**2)
        feats[f'wavelet_energy_post_l{i}'] = np.sum(c_post**2)
    return feats

## 5. Feature Engineering - Change-Point Detection
Use ruptures to detect change points and extract algorithmic features.

In [ ]:
# Feature Engineering - Change-Point Detection
def extract_cp_features(value, boundary):
    algo = rpt.Pelt(model='rbf').fit(value)
    bkps = algo.predict(pen=10)
    closest_cp = min(bkps, key=lambda x: abs(x-boundary)) if bkps else -1
    closest_cp_dist = abs(closest_cp-boundary)
    is_boundary_detected = int(closest_cp_dist == 0)
    cp_strength = algo.cost.sum_of_costs(bkps)
    return {
        'cp_closest_dist': closest_cp_dist,
        'cp_boundary_detected': is_boundary_detected,
        'cp_strength': cp_strength
    }

## 6. Feature Engineering - Automated Features
Apply catch22 and selective tsfresh features for automated time series feature extraction.

In [ ]:
# Feature Engineering - Automated Features
def extract_auto_features(x):
    catch22_feats = pycatch22.catch22_all(x)
    return {f'catch22_{k}': v for k, v in catch22_feats.items()}
# tsfresh omitted for speed; add if needed

## 7. Feature Selection and Storage
Remove constant features, handle correlations, apply feature selection, and save processed features.

In [ ]:
# Feature Selection and Storage
def select_features(X, y=None):
    vt = VarianceThreshold()
    X_vt = vt.fit_transform(X)
    keep = vt.get_support()
    X_sel = X.loc[:, keep]
    # Optionally: remove correlated features, use SelectFromModel
    return X_sel

def save_features(features_df, path):
    features_df.to_parquet(path)

## 8. Tabular Model Training (LightGBM)
Train LightGBM models with GroupKFold cross-validation and generate out-of-fold predictions.

In [ ]:
# Tabular Model Training (LightGBM)
def train_tabular(X, y, groups, output_dir):
    lgb_params = {
        'objective': 'binary', 'metric': 'auc', 'boosting_type': 'gbdt',
        'learning_rate': 0.03, 'num_leaves': 128, 'min_data_in_leaf': 50,
        'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 5,
        'lambda_l1': 0.5, 'lambda_l2': 1.0, 'n_jobs': 8, 'seed': SEED
    }
    oof_preds = np.zeros(len(y))
    models = []
    cv = GroupKFold(n_splits=5)
    for fold, (tr, va) in enumerate(cv.split(X, y, groups)):
        train_set = lgb.Dataset(X.iloc[tr], y.iloc[tr])
        val_set = lgb.Dataset(X.iloc[va], y.iloc[va])
        model = lgb.train(lgb_params, train_set, num_boost_round=5000,
                          valid_sets=[val_set], early_stopping_rounds=200, verbose_eval=False)
        oof_preds[va] = model.predict(X.iloc[va])
        models.append(model)
    joblib.dump(models, os.path.join(output_dir, 'models/lgb_model.pkl'))
    pd.Series(oof_preds, index=y.index).to_parquet(os.path.join(output_dir, 'oof_preds.parquet'))
    return models, oof_preds

## 9. Deep Learning Model Implementation
Implement the two-branch CNN architecture for pre/post segments, including data loaders and training loop.

In [ ]:
# Deep Learning Model Implementation
class ConvEncoder(nn.Module):
    def __init__(self, in_ch=1, base=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(in_ch, base, kernel_size=7, padding=3),
            nn.BatchNorm1d(base), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(base, base*2, kernel_size=5, padding=2),
            nn.BatchNorm1d(base*2), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(base*2, base*4, kernel_size=3, padding=1),
            nn.BatchNorm1d(base*4), nn.ReLU(), nn.AdaptiveAvgPool1d(1),
            nn.Flatten(), nn.Linear(base*4, 128), nn.ReLU()
        )
    def forward(self, x):
        return self.net(x)

class TwoBranchCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.pre_enc = ConvEncoder()
        self.post_enc = ConvEncoder()
        self.classifier = nn.Sequential(
            nn.Linear(256, 128), nn.ReLU(), nn.Dropout(0.4),
            nn.Linear(128, 1)
        )
    def forward(self, x_pre, x_post):
        e1 = self.pre_enc(x_pre)
        e2 = self.post_enc(x_post)
        e = torch.cat([e1, e2], dim=1)
        return self.classifier(e).squeeze(1)

# Training loop and DataLoader omitted for brevity; see plan for details

## 10. Cross-Validation Strategy
Implement GroupKFold validation, compute ROC AUC, and generate OOF predictions for stacking.

In [ ]:
# Cross-Validation Strategy
def cross_validate(X, y, groups, model_fn):
    cv = GroupKFold(n_splits=5)
    oof_preds = np.zeros(len(y))
    for fold, (tr, va) in enumerate(cv.split(X, y, groups)):
        model = model_fn()
        # Fit model on X.iloc[tr], y.iloc[tr]; predict on X.iloc[va]
        # For LightGBM, use train_tabular; for CNN, use custom loop
        # oof_preds[va] = model.predict(X.iloc[va])
    return oof_preds

## 11. Ensemble and Stacking
Combine base model predictions using meta-learning and weighted averaging.

In [ ]:
# Ensemble and Stacking
def train_stacker(oof_tabular, oof_cnn, y, output_dir):
    meta_X = np.vstack([oof_tabular, oof_cnn]).T
    meta_model = LogisticRegression(C=1.0, max_iter=1000)
    meta_model.fit(meta_X, y)
    joblib.dump(meta_model, os.path.join(output_dir, 'models/meta_model.pkl'))
    return meta_model

def predict_stacker(tabular_preds, cnn_preds, meta_model):
    meta_X = np.vstack([tabular_preds, cnn_preds]).T
    return meta_model.predict_proba(meta_X)[:,1]

## 12. Model Artifacts Management
Save and load trained models, feature extractors, scalers, and metadata for reproducible inference.

In [ ]:
# Model Artifacts Management
def save_model(model, path):
    joblib.dump(model, path)
def load_model(path):
    return joblib.load(path)

-----------------

## 13. Train Function Implementation
Implement the required train(X_train, y_train, output_dir) function.

In [ ]:
# Train Function Implementation
def train(X_train, y_train, output_dir):
    os.makedirs(os.path.join(output_dir, 'models'), exist_ok=True)
    # 1. Preprocess and extract features
    series_dict, _ = read_dataset(X_train)
    features = {}
    for id_, series in tqdm(series_dict.items()):
        proc = preprocess_series(series)
        feats = extract_stat_features(proc['x_pre'], proc['x_post'], proc['x_pre_dt'], proc['x_post_dt'])
        feats.update(extract_cp_features(series['value'], np.argmax(series['period']==1)))
        feats.update(extract_auto_features(series['value']))
        feats['len_pre'] = proc['len_pre']
        feats['len_post'] = proc['len_post']
        feats['n_missing_pre'] = proc['n_missing_pre']
        feats['n_missing_post'] = proc['n_missing_post']
        features[id_] = feats
    features_df = pd.DataFrame.from_dict(features, orient='index')
    features_df = select_features(features_df, y_train)
    save_features(features_df, os.path.join(output_dir, 'features.parquet'))
    # 2. Train tabular model
    models, oof_tabular = train_tabular(features_df, y_train, features_df.index, output_dir)
    # 3. Train CNN model (omitted for brevity)
    oof_cnn = np.zeros_like(oof_tabular) # Placeholder
    # 4. Train stacker
    meta_model = train_stacker(oof_tabular, oof_cnn, y_train, output_dir)

## 14. Infer Function Implementation
Implement the required infer(X_test, output_dir) function.

In [ ]:
# Infer Function Implementation
def infer(X_test, output_dir):
    series_dict, _ = read_dataset(X_test)
    features = {}
    for id_, series in tqdm(series_dict.items()):
        proc = preprocess_series(series)
        feats = extract_stat_features(proc['x_pre'], proc['x_post'], proc['x_pre_dt'], proc['x_post_dt'])
        feats.update(extract_cp_features(series['value'], np.argmax(series['period']==1)))
        feats.update(extract_auto_features(series['value']))
        feats['len_pre'] = proc['len_pre']
        feats['len_post'] = proc['len_post']
        feats['n_missing_pre'] = proc['n_missing_pre']
        feats['n_missing_post'] = proc['n_missing_post']
        features[id_] = feats
    features_df = pd.DataFrame.from_dict(features, orient='index')
    # Feature selection (use train mask)
    train_feats = pd.read_parquet(os.path.join(output_dir, 'features.parquet'))
    features_df = features_df[train_feats.columns]
    # Load models
    models = load_model(os.path.join(output_dir, 'models/lgb_model.pkl'))
    tabular_preds = np.mean([m.predict(features_df) for m in models], axis=0)
    cnn_preds = np.zeros_like(tabular_preds) # Placeholder
    meta_model = load_model(os.path.join(output_dir, 'models/meta_model.pkl'))
    final_preds = predict_stacker(tabular_preds, cnn_preds, meta_model)
    return pd.Series(final_preds, index=features_df.index)

----------------------

### The `train()` Function

In this function, you build and train your model for making inferences on the test data. Your model must be stored in the `model_directory_path`.

The baseline implementation below doesn't require a pre-trained model, as it uses a statistical test that will be computed at inference time.

In [ ]:
def train(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    model_directory_path: str,
):
    # For our baseline t-test approach, we don't need to train a model
    # This is essentially an unsupervised approach calculated at inference time
    model = None

    # You could enhance this by training an actual model, for example:
    # 1. Extract features from before/after segments of each time series
    # 2. Train a classifier using these features and y_train labels
    # 3. Save the trained model

    joblib.dump(model, os.path.join(model_directory_path, 'model.joblib'))

### The `infer()` Function

In the inference function, your trained model (if any) is loaded and used to make predictions on test data.

**Important workflow:**
1. Load your model;
2. Use the `yield` statement to signal readiness to the runner;
3. Process each dataset one by one within the for loop;
4. For each dataset, use `yield prediction` to return your prediction.

**Note:** The datasets can only be iterated once!

In [ ]:
def infer(
    X_test: typing.Iterable[pd.DataFrame],
    model_directory_path: str,
):
    model = joblib.load(os.path.join(model_directory_path, 'model.joblib'))

    yield  # Mark as ready

    # X_test can only be iterated once.
    # Before getting the next dataset, you must predict the current one.
    for dataset in X_test:
        # Baseline approach: Compute t-test between values before and after boundary point
        # The negative p-value is used as our score - smaller p-values (larger negative numbers)
        # indicate more evidence against the null hypothesis that distributions are the same,
        # suggesting a structural break
        def t_test(u: pd.DataFrame):
            return -scipy.stats.ttest_ind(
                u["value"][u["period"] == 0],  # Values before boundary point
                u["value"][u["period"] == 1],  # Values after boundary point
            ).pvalue

        prediction = t_test(dataset)
        yield prediction  # Send the prediction for the current dataset

        # Note: This baseline approach uses a t-test to compare the distributions
        # before and after the boundary point. A smaller p-value (larger negative number)
        # suggests stronger evidence that the distributions are different,
        # indicating a potential structural break.

## Local testing

To make sure your `train()` and `infer()` function are working properly, you can call the `crunch.test()` function that will reproduce the cloud environment locally. <br />
Even if it is not perfect, it should give you a quick idea if your model is working properly.

In [ ]:
crunch.test(
    # Uncomment to disable the train
    # force_first_train=False,

    # Uncomment to disable the determinism check
    # no_determinism_check=True,
)

## Results

Once the local tester is done, you can preview the result stored in `data/prediction.parquet`.

In [ ]:
prediction = pd.read_parquet("data/prediction.parquet")
prediction

### Local scoring

You can call the function that the system uses to estimate your score locally.

In [ ]:
# Load the targets
target = pd.read_parquet("data/y_test.reduced.parquet")["structural_breakpoint"]

# Call the scoring function
sklearn.metrics.roc_auc_score(
    target,
    prediction,
)

# Submit your Notebook

To submit your work, you must:
1. Download your Notebook from Colab
2. Upload it to the platform
3. Create a run to validate it

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Download and Submit Notebook](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/download-and-submit-notebook.gif)